# First bar candlestick color & day candlestick color - Analytics

### Import Library

In [1]:
import numpy as np
import pandas as pd
import numpy as np
import pandas_ta as ta
import seaborn as sns
import math

import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

### Load Price Data

In [2]:
import os
from pathlib import Path
notebook_path = os.getcwd()
algo_dir = Path(notebook_path).parent.parent
csv_file = str(algo_dir) + '/vn-stock-data/VN30ps/VN30F1M_5minutes.csv'
is_file = os.path.isfile(csv_file)
if is_file:
    dataset = pd.read_csv(csv_file, index_col='Date', parse_dates=True)
else:
    print('remote')
    dataset = pd.read_csv("https://raw.githubusercontent.com/zuongthaotn/vn-stock-data/main/VN30ps/VN30F1M_5minutes.csv", index_col='Date', parse_dates=True)

In [3]:
data = dataset.copy()

In [4]:
def bar9h(tick):
    tick=tick[(tick.index.hour == 9) & (tick.index.minute == 0)]
    if len(tick):
        return tick[0]

data['Close_9h'] = data['Close']
day_data = data.resample('D').agg({
        'Open': 'first',
        'Close': 'last',
        'High': 'max',
        'Low': 'min',
        'Close_9h': bar9h
    })
day_data.dropna(subset=['Close'], inplace=True)

In [5]:
def get_candlestick_color(open_price, close_price):
    return 'doji' if open_price == close_price else ('white' if open_price < close_price else 'black')
day_data['first_bar_color'] = day_data.apply(lambda r: get_candlestick_color(r['Open'], r['Close_9h']), axis=1)
day_data['day_color'] = day_data.apply(lambda r: get_candlestick_color(r['Open'], r['Close']), axis=1)

In [6]:
day_data[day_data.first_bar_color == day_data.day_color]

,Open,Close,High,Low,Close_9h,first_bar_color,day_color
Date,,,,,,,
2018-08-20,946.1,945.0,951.5,942.6,945.8,black,black
2018-08-22,959.5,957.0,960.6,953.6,959.3,black,black
2018-08-24,959.5,958.5,963.0,956.4,959.1,black,black
2018-08-27,960.8,963.2,965.2,960.7,961.5,white,white
2018-08-28,965.9,965.6,967.6,964.4,965.7,black,black
...,...,...,...,...,...,...,...
2024-07-17,1309.0,1303.2,1321.8,1294.7,1308.2,black,black
2024-07-19,1304.6,1297.1,1311.2,1295.3,1304.4,black,black
2024-07-22,1291.3,1300.0,1304.4,1291.3,1293.6,white,white


In [7]:
len(day_data)

1490

In [8]:
day_data.groupby(day_data.index.year).count()

,Open,Close,High,Low,Close_9h,first_bar_color,day_color
Date,,,,,,,
2018,98,98,98,98,98,98,98
2019,250,250,250,250,248,250,250
2020,249,249,249,249,247,249,249
2021,250,250,250,250,250,250,250
2022,249,249,249,249,249,249,249
2023,249,249,249,249,248,249,249
2024,145,145,145,145,145,145,145


In [9]:
day_data[(day_data.first_bar_color != day_data.day_color) & (day_data.first_bar_color != 'doji')]

,Open,Close,High,Low,Close_9h,first_bar_color,day_color
Date,,,,,,,
2018-08-13,943.5,954.2,954.6,942.3,943.1,black,white
2018-08-14,955.5,959.3,961.8,952.1,954.9,black,white
2018-08-15,958.0,946.6,962.3,946.6,958.7,white,black
2018-08-16,942.4,947.0,947.3,935.5,942.0,black,white
2018-08-17,952.2,947.0,954.6,945.0,953.0,white,black
...,...,...,...,...,...,...,...
2024-07-24,1283.3,1283.5,1287.4,1273.3,1282.7,black,white
2024-07-26,1278.7,1285.0,1285.0,1276.8,1278.3,black,white
2024-07-29,1289.9,1287.8,1294.7,1287.8,1291.2,white,black
